In [1]:
from fnl_pipe.cmb import ACTPipe, ACTMetadata
from fnl_pipe.util import OutputManager
from fnl_pipe.pipe import GalPipe

from pixell import enmap, enplot
from pixell.curvedsky import alm2map, map2alm, alm2cl, almxfl

import matplotlib.pyplot as plt
import numpy as np


def has_nan(ar):
    return np.isnan(np.sum(ar))


def get_size_alm(lmax):
    return int((lmax**2 + 3*lmax)/2 + 1)


# def get_tt_inv(cl_tt):
#     assert len(cl_tt.shape) == 3
#     assert cl_tt.shape[0] == cl_tt.shape[1]
    
# #     nfreq = cl_tt.shape[0]
# #     assert nfreq <= 3
    
# # #     cl_inv = np.empty(cl_tt.shape, dtype=np.float64)
# #     if nfreq == 1:
# #         cl_inv = 1./cl_tt
# #     elif nfreq == 2:
# #         det = cl_tt[0,0] * cl_tt[1,1] - cl_tt[0,1] * cl_tt[1,0]
# #         assert not np.any(det == 0.)
# #         cl_inv = np.array([[cl_tt[1,1], -cl_tt[0,1]],
# #                            [-cl_tt[1,0], cl_tt[0,0]]])/det
# #     elif nfreq == 3:
        
        
# #     return cl_inv

#     return np.linalg.inv(cl_tt.T)
    

def dot(*args):
    for arg in args:
        print('arg shape: ' + str(arg.shape))
    nargs = len(args)
    assert nargs >= 2 
    if nargs == 2:
        ret = np.dot(args[0], args[1])
        print('ret' + str(ret.shape))
        return np.dot(args[0], args[1])
    else:
        print(len(args[1:]))
        ret = np.dot(args[0], dot(*args[1:]))
        print('ret' + str(ret.shape))
        return np.dot(args[0], dot(*args[1:]))
    

def matmul(*args):
    nargs = len(args)
    assert nargs >= 2 
    if nargs == 2:
        return np.matmul(args[0], args[1])
    else:
        return np.matmul(args[0], matmul(*args[1:]))


class ACTMultiFreqPipe:
    def __init__(self, pipes, l_ksz_sum=2000):
        self.pipes = pipes
        freqs = [pipe.freq for pipe in pipes]
        assert freqs == sorted(freqs)
#         assert 150 in freqs
        
        self.freqs = freqs
        
        self.nfreq = len(freqs)
        
        self.pipe_map = {}
        for freq, pipe in zip(freqs, self.pipes):
            self.pipe_map[freq] = pipe
        
#         self.pipe_150 = self.pipe_map[150]
        self.pipe_150 = self.pipe_map[freqs[0]]
        self.lmax = self.pipes[0].lmax
        self.l_ksz_sum = l_ksz_sum
        
        self.step_lweight = np.zeros(self.lmax + 1) # a default l-weight applied to each individual frequency
        self.step_lweight[1500:] = 1.
        
        ells = np.arange(self.lmax + 1)
        r_lwidth = self.pipe_150.metadata.r_lwidth
        
        self.cl_th = np.exp(-(ells / r_lwidth / 5500.)**2)
        self.cl_zz = np.ones(self.lmax + 1)
        
        self.init_data = False
        self.init_fkp_f = False
        self.init_t_tilde = False
        self.init_cl_tt = False
        self.init_inv = False
        
    def import_data(self):
        beams = []
        for pipe in self.pipes:
            if not pipe.init_data:
                pipe.import_data()
            beams.append(pipe.beam)    
        self.beams = np.array(beams)
        
        self.bl = np.zeros((self.nfreq, self.nfreq, self.lmax + 1))
        self.bl_inv = np.zeros((self.nfreq, self.nfreq, self.lmax + 1))
        for ifreq in range(self.nfreq):
            self.bl[ifreq, ifreq] = self.beams[ifreq]
            self.bl_inv[ifreq, ifreq] = 1./self.beams[ifreq]
        
        self.map_t_150 = self.pipe_150.map_t
        self.init_data = True
        
    def init_fkp(self):
        assert self.init_data
        
        pipe_150 = self.pipe_150
        b2_l0 = pipe_150.beam[1]**2
        r_fkp = pipe_150.metadata.r_fkp
        ctt_3k_act = 24.8 * 2 * np.pi / 3000 / (3000 + 1)

        a0 = r_fkp / b2_l0 / ctt_3k_act

        onemap = pipe_150.make_zero_map() + 1.
        
        eta2_prod = onemap.copy()

        eta2s = []
        for freq, pipe in self.pipe_map.items():
            this_eta = pipe.ivar_t / pipe.pixel_area
            eta2_prod *= this_eta
            eta2s.append(this_eta)

        eta2_sum = pipe_150.make_zero_map()
        for i in range(len(pipes)):
            this_prod = onemap.copy()
            for j in range(len(pipes)):
                if j!= i:
                    this_prod *= eta2s[j]

            eta2_sum += this_prod
        
        nan_mask = eta2_prod == 0.
        self.comb_fkp = eta2_prod / (eta2_prod + eta2_sum * a0)
        self.comb_fkp[nan_mask] = 0.
        assert not has_nan(self.comb_fkp)

        print(f'combined fkp min/max: {np.min(self.comb_fkp):.3e} {np.max(self.comb_fkp):.3e}')
        
        fig = enplot.plot(enmap.downgrade(self.comb_fkp, 32), ticks=15, colorbar=True)
        om.savefig('combined_fkp', mode='pixell', fig=fig)
        
        self.init_fkp_f = True
        
    def set_fkp(self, fkp_weight):
        assert self.init_data
#         assert self.map_t_150.wcs == fkp_weight.wcs # why doesn't this work?
        assert self.map_t_150.shape == fkp_weight.shape
        
        self.comb_fkp = fkp_weight
        self.init_fkp_f = True
        
    def process_t(self):
        assert self.init_data
        assert self.init_fkp_f
        
        t_tilde = enmap.ndmap(np.empty((self.nfreq, get_size_alm(self.lmax)), dtype=np.complex64), 
                              self.pipe_150.map_t.wcs)
        for ifreq in range(self.nfreq):
            pipe = pipes[ifreq]
            t_fkp = pipe.map_t * self.comb_fkp
            t_tilde_lm = almxfl(map2alm(t_fkp, lmax=self.lmax),lfilter=self.step_lweight)
            t_tilde[ifreq,:] = t_tilde_lm
             
        self.t_tilde = t_tilde
        self.init_t_tilde = True
    
    def compute_cl_tt(self, compute_inverse=True):
        assert self.init_t_tilde
        t_tilde = self.t_tilde
        cl_tt = alm2cl(t_tilde[:,None,:], t_tilde[None,:,:])
        self.cl_tt = cl_tt
        
        if compute_inverse:
#             for l in range(self.l_ksz_sum, self.lmax + 1, 1):
#                 print(f'inverting cl_tt matrix l={l}')
#                 print(cl_tt[:,:,l])
#                 cl_tt_inv = np.linalg.inv(cl_tt[:,:,l])
#                 print(cl_tt_inv)
            cl_tt_inv = np.linalg.inv(cl_tt[...,self.l_ksz_sum:].T).T
            self.cl_tt_inv = cl_tt_inv
            self.init_inv = True
        
        self.init_cl_tt = True

    def compute_var(self):
        assert self.init_inv
        assert self.init_cl_tt
        
        ells = np.arange(self.lmax + 1)[self.l_ksz_sum:]
        cl_th = self.cl_th[self.l_ksz_sum:]
        bl = self.bl[...,self.l_ksz_sum:].T
        bl_inv = self.bl_inv[...,self.l_ksz_sum:].T
        
        cl_inv = self.cl_tt_inv.T
        cl_tt = self.cl_tt[...,self.l_ksz_sum:].T
        
        bcb = matmul(bl, cl_inv, bl)
    
        summand = (2 * ells[:, None, None] + 1) * cl_th[:, None, None] ** 2 * bcb
        self.var = 1./(summand.sum())

        return self.var


class FreqFormatStr:
    def __init__(self, a,b):
        self.a = a
        self.b = b
    
    def get_str(self, freq):
        fstring = str(freq)
        if len(fstring) < 3:
            fstring = '0' + fstring
            
        return self.a + fstring + self.b
    

In [4]:
data_path = '/home/aroman/data/'

planck_path = data_path + 'planck/'
mask_path = data_path + 'mask/'
pipe_path = data_path + 'pipe/'

act_path = data_path + 'act/'

map_path = FreqFormatStr(act_path + 'act_planck_s08_s19_cmb_f','_daynight_srcfree_map.fits')
ivar_path = FreqFormatStr(act_path + 'act_planck_s08_s19_cmb_f','_daynight_srcfree_ivar.fits')
beam_path = FreqFormatStr(act_path + 'beam_f','_daynight.txt') # proprietary beam file

catalog_path = data_path + 'vr_summaries/v01_sdss_cmass_north.h5'

planck_enmap_path = mask_path + 'planck_foreground.npy'

freqs = [90, 150, 220]

om = OutputManager(base_path='output', title='multifreq_variance_nb')
act_md = ACTMetadata(r_fkp=1.56, r_lwidth=0.62)

pipe_map = {}
pipes = []
for freq in freqs:
    this_pipe = ACTPipe(map_path.get_str(freq), ivar_path.get_str(freq), beam_path.get_str(freq), planck_enmap_path,
                           om, freq=freq, metadata=act_md, plots=True, lmax=12000)
    pipes.append(this_pipe)
    pipe_map[freq] = this_pipe
    
multi_pipe = ACTMultiFreqPipe(pipes)
multi_pipe.import_data()
multi_pipe.init_fkp()

comb_fkp = multi_pipe.comb_fkp

importing act map /home/aroman/data/act/act_planck_s08_s19_cmb_f090_daynight_srcfree_map.fits
importing act ivar map /home/aroman/data/act/act_planck_s08_s19_cmb_f090_daynight_srcfree_ivar.fits
importing beam /home/aroman/data/act/beam_f090_daynight.txt
importing act map /home/aroman/data/act/act_planck_s08_s19_cmb_f150_daynight_srcfree_map.fits
importing act ivar map /home/aroman/data/act/act_planck_s08_s19_cmb_f150_daynight_srcfree_ivar.fits
importing beam /home/aroman/data/act/beam_f150_daynight.txt
importing act map /home/aroman/data/act/act_planck_s08_s19_cmb_f220_daynight_srcfree_map.fits
importing act ivar map /home/aroman/data/act/act_planck_s08_s19_cmb_f220_daynight_srcfree_ivar.fits
importing beam /home/aroman/data/act/beam_f220_daynight.txt


/tmp/ipykernel_98583/723524614.py:150: RuntimeWarning: invalid value encountered in true_divide
  self.comb_fkp = eta2_prod / (eta2_prod + eta2_sum * a0)


combined fkp min/max: 0.000e+00 2.001e-01


In [5]:
freqs = [90, 150, 220]

pipes = [pipe_map[freq] for freq in freqs]

multi_pipe = ACTMultiFreqPipe(pipes)
multi_pipe.import_data()
multi_pipe.set_fkp(comb_fkp)

gal_pipe = GalPipe(catalog_path, pipes[0].map_t, diag_plots=True)
gal_pipe.import_data()
gal_pipe.make_vr_list()
gal_pipe.make_vr_map()

importing galaxy catalog /home/aroman/data/vr_summaries/v01_sdss_cmass_north.h5
done


In [6]:
z_alm = map2alm(gal_pipe.vr_map, lmax=multi_pipe.lmax)
cl_zz = alm2cl(z_alm)
cl_zz[:1000] = np.nan

ells = np.arange(multi_pipe.lmax + 1)
plt.figure(dpi=300)
plt.plot(ells, cl_zz)
plt.ylabel(r'$C_l^{ZZ}$')
plt.xlabel(r'$l$')
om.savefig('cl_zz', mode='matplotlib')
plt.close()

In [7]:
multi_pipe.process_t()
multi_pipe.compute_cl_tt()
var = multi_pipe.compute_var()

print(np.sqrt(var))
    
# for ifreq in range(len(freqs)):
#     names = 
    
# names = ['90-90','90-150','150-150']
# cl_tt = multi_pipe.cl_tt.copy()
# cls = [cl_tt[0,0,:], cl_tt[0,1,:], cl_tt[1,1,:]]

# plt.figure(dpi=300)
# for icl in range(3):
#     this_cl = cls[icl]
#     this_cl[:1500] = np.nan
#     plt.plot(ells, this_cl, label=names[icl])
# plt.legend()
# om.savefig('cl_tt', mode='matplotlib')
# plt.close()



1.7927609165372786e-07
